<a href="https://colab.research.google.com/github/codebuzer/Bioactivity-Prediction-App/blob/main/pysam_need_parallelize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pysam
!apt-get install samtools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
samtools is already the newest version (1.10-3).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:
import pysam
import pandas as pd
import numpy as np
import subprocess
import time

In [ ]:
df.reset_index().to_feather('gtffile.ftr')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/gene wise coverage/gtffile.csv', index_col = 0, low_memory = False)

In [ ]:
df.head()

,Region,Source,Region_name,Start,End,Description,Region_length
0,1,ensembl_havana,gene,1211340,1214153,"gene_id ""ENSG00000186827""; gene_version ""11""; ...",2813
1,1,ensembl_havana,transcript,1211340,1214153,"gene_id ""ENSG00000186827""; gene_version ""11""; ...",2813
2,1,ensembl_havana,exon,1213983,1214153,"gene_id ""ENSG00000186827""; gene_version ""11""; ...",170
3,1,ensembl_havana,CDS,1213983,1214127,"gene_id ""ENSG00000186827""; gene_version ""11""; ...",144
4,1,ensembl_havana,start_codon,1214125,1214127,"gene_id ""ENSG00000186827""; gene_version ""11""; ...",2


In [ ]:
def calculate_gene_coverage(bam_file, df):
    total_time_start = time.time()
    print("Indexing BAM file...")
    index_start_time = time.time()
    subprocess.run(["samtools", "index", bam_file])
    index_end_time = time.time()
    print('Indexing took', index_end_time - index_start_time, "Seconds")
    print('Smile Indexing Done...')
    print("Now  Aligning BAM file...")
    bam_proc_start = time.time()
    bam = pysam.AlignmentFile(bam_file, "rb")
    bam_proc_end = time.time()
    print('BAM aligning took',bam_proc_end - bam_proc_start, 'seconds' )
    print('Now go and get some coffee or tea because this step to calculate the coverage takes time...')
    coverage_data = []
    start_count_coverage = time.time()
    
    
    for _, gene in df.iterrows():
        chrom = gene['Region']
        start = gene['Start']
        end = gene['End']
        region = end - start + 1
        coverage = bam.count_coverage(chrom, start, end)
        
        covbases = sum(sum(depth >= 1 for depth in depth_array) for depth_array in coverage)
        covpercent = (covbases / region) * 100

        gene_name = gene['Description'].split(";")[0].split(" ")[1].replace('"', '')
        
        # Adjust covbases and covpercent if necessary
        covbases = np.where(covbases > region, region, covbases)
        covpercent = np.where(covpercent > 100, 100.0, covpercent)
        
        coverage_data.append((gene_name, chrom, start, end, covbases, covpercent))
    end_count_coverage = time.time()
    print('Counting coverage took', end_count_coverage - start_count_coverage, 'seconds')
        
    df = pd.DataFrame(coverage_data, columns=['Gene', 'Chromosome', 'Start', 'End', 'CovBases', 'CovPercent'])
    bam.close()
    
    print("Gene coverage calculation completed.")
    total_time_end = time.time()
    print("Total time taken to execute this entire process :", total_time_end - total_time_start, 'seconds')


    return df

bam_file = "/content/drive/MyDrive/gene wise coverage/IGIB1130527553V.bam"

result_df = calculate_gene_coverage(bam_file, df)


Indexing BAM file...
Indexing took 32.57274651527405 Seconds
Smile Indexing Done...
Now  Aligning BAM file...
BAM aligning took 0.2102658748626709 seconds
Now go and get some coffee or tea because this step to calculate the coverage takes time...


In [ ]:
result_df.to_csv('IGIB113023885V_whole_gene_wise_coverage.csv')

In [ ]:
import pysam

print(pysam.__version__)

0.21.0
